# ✍ Aufgabe

- Wir wollen unsere Sensorklasse aus einer der vergangenen Einheiten erweitern.
- Unsere Sensoren bekommen ein Kalibrierungsdatum &rarr; wir wollen nun in der Lage sein unsere Sensoren anhand dieses Merkmales zu sortieren
- Wir wollen unsere Sensoren auch serialisieren können &rarr; wir nutzen JSON als Format

In [10]:
from datetime import datetime
import json

## Grundlegendes zur Serialisierung

### Dictionarys und Listen

Dictionarys und Listen können in Python einfach in JSON-Strings umgewandelt werden. Dazu gibt es die Funktionen `json.dumps()` und `json.loads()`. Die Funktion `json.dumps()` wandelt ein Dictionary oder eine Liste in einen JSON-String um. Die Funktion `json.loads()` wandelt einen JSON-String in ein Dictionary oder eine Liste um.


In [11]:
my_dict = {"A": "T", "T": "A", "G": "C","C" : "G"}

json_string = json.dumps(my_dict, indent=4)

print(json_string)

{
    "A": "T",
    "T": "A",
    "G": "C",
    "C": "G"
}


In [12]:
string_to_json = json.loads(json_string)
print(string_to_json)

{'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}


Anstelle von `json.loads()` kann auch `json.load()` verwendet werden, um direkt aus einer Datei zu lesen. Analog dazu gibt es auch `json.dump()`.

In [13]:
with open("data.json", "w") as f:
    json.dump(my_dict, indent=4, fp=f)

In [14]:
with open("data.json", "r") as f:
    my_dict = json.load(f)

my_dict

{'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}

In [15]:
class my_nucleotides_dict(dict):
    def __init__(self):
        self = {"A": "T", "T": "A", "G": "C","C" : "G"}
    def __str__(self):
        return json.dumps(self, indent=4)

### Objekte

Objekte können nicht direkt in JSON umgewandelt werden. Dazu müssen wir die `__dict__`-Methode verwenden, um ein Dictionary zu erhalten, das wir dann in einen JSON-String umwandeln können. Sofern alle Attribute primitive Datentypen sind geschieht dies automatisch.

In [16]:
class Sensor():

    def __init__(self, id: int, sens_type: str, sensitivity: float = 1.0):
        self.id = id
        self.sens_type = sens_type
        self.sensitivity = sensitivity

s1 = Sensor(1, "Temperature")

print(s1.__dict__)

{'id': 1, 'sens_type': 'Temperature', 'sensitivity': 1.0}


In [17]:
class DataPoint():
    def __init__(self, value, timestamp):
        self.value = value
        self.timestamp = timestamp

class SensorWithData(Sensor):
    def __init__(self, id: int, sens_type: str, sensitivity: float, data: DataPoint):
        super().__init__(id, sens_type, sensitivity)
        self.data = data

d1 = DataPoint(23, "21.02.2000")

s2 = SensorWithData(2, "Humidity", 3.0, d1)

Sind die Attribute jedoch komplexer, so müssen wir diese rekursiv in ein Dictionary umwandeln. 

In [18]:
s2.__dict__

{'id': 2,
 'sens_type': 'Humidity',
 'sensitivity': 3.0,
 'data': <__main__.DataPoint at 0x7f02c1b10080>}

Dies kann beispielsweise mit der Funktion `json.dumps()` geschehen die in `__repr__`-Methode aufgerufen wird.

In [19]:
class DataPoint():
    def __init__(self, value, timestamp):
        self.value = value
        self.timestamp = timestamp
    def __repr__(self) -> str:
        return json.dumps(self.__dict__)

class SensorWithData(Sensor):
    def __init__(self, id: int, sens_type: str, sensitivity: float, data: DataPoint):
        super().__init__(id, sens_type, sensitivity)
        self.data = data

    def __str__(self):
        return {"id": self.id, "sens_type": self.sens_type, "sensitivity": self.sensitivity, "data": self.data.__repr__()}

In [16]:
d2 = DataPoint(23, "21.02.2000")

s3 = SensorWithData(2, "Humidity", 3.0, d2)

s3.__dict__

{'id': 2,
 'sens_type': 'Humidity',
 'sensitivity': 3.0,
 'data': {"value": 23, "timestamp": "21.02.2000"}}

# Erweiterung unserer Sensor-Klasse

In [17]:
class Sensor:
    """
    A class representing a sensor with a numerical ID, a type, a sensitivity, and a history of measurements.

    Attributes:
        id (int): The numerical ID of the sensor.
        sens_type (str): The name of the sensor.
        sensitivity (float): The sensitivity of the sensor
        calibration_date (datetime): The date of the last calibration.
        last_measurement (float): The last measured value.
        measurement_history (List[float]): A list of the last 10 measured values.
    """

    def __init__(self, id: int, sens_type: str, sensitivity: float, calibration_date: datetime):
        """
        Initializes a new Sensor instance.

        Args:
            id (int): The numerical ID of the sensor.
            sens_type (str): The name of the sensor.
            sensitivity (float): The sensitivity of the sensor
            calibration_date (datetime): The date of the last calibration.
        """
        self.id = id
        self.sens_type = sens_type
        self.sensitivity = sensitivity
        self.calibration_date = calibration_date
        self.__last_measurement = 0.0
        self.measurement_history = []

    def set_last_measurement(self, measurement: float) -> None:
        """
        Sets the last measurement value and adds it to the measurement history.

        Args:
            measurement (float): The new measurement value.
        """
        self.__last_measurement = measurement
        self.measurement_history.append(measurement)
        if len(self.measurement_history) > 10:
            self.measurement_history.pop(0)

    def calculate_mean(self) -> float:
        return sum(self.measurement_history) / len(self.measurement_history)

    def print_mean(self) -> None:
        """
        Prints the mean of all measurements in the history.
        """
        if len(self.measurement_history) == 0:
            print(f"No measurements available for sensor '{self.sens_type}' (ID: {self.id}).")
        else:
            mean = self.calculate_mean()
            print(f"Mean measurement for sensor '{self.sens_type}' (ID: {self.id}): {mean:.2f}")

    def __str__(self) -> str:
        return F"Sensor '{self.sens_type}' (ID: {self.id}) was calibrated on {self.calibration_date.strftime('%d.%m.%Y')}."
    
    def __repr__(self) -> str:
        return self.__str__()


## Wir wollen unsere Sensoren anhand des Kalibrierungsdatums sortieren können
Standardmäßig sind unsere selbst erstellen Klassen nicht mit der normalen `sort`-Methode sortierbar.

In [18]:
#%%
sensor_1 = Sensor(1, "Temperature", 1.0, datetime(2022, 1, 1))
sensor_2 = Sensor(2, "Pressure", 3.0, datetime(2023, 8, 17))
sensor_3 = Sensor(3, "Humidity", 5.0, datetime(2021, 10, 31))

#%%
# Create a list of sensors that should be sorted by calibration date
my_sensors = [sensor_1, sensor_2, sensor_3]
my_sensors.sort()

for my_sensor in my_sensors:
    print(my_sensor)

TypeError: '<' not supported between instances of 'Sensor' and 'Sensor'

Wir erweitern unsere Klasse um eine Methode `__lt__` (less than), die zwei Sensoren vergleicht und `True` zurückgibt, wenn der erste Sensor kleiner als der zweite ist.  
Sobald wir unsere Sensoren vergleichbar gemacht haben, können diese auch sortiert werden.

In [19]:
class SortableSensor(Sensor):
    # dunder method needed to make object sortable
    def __lt__(self, other):
        return self.calibration_date < other.calibration_date
    

sensor_1 = SortableSensor(1, "Temperature", 1.0, datetime(2022, 1, 1))
sensor_2 = SortableSensor(2, "Pressure", 3.0, datetime(2023, 8, 17))
sensor_3 = SortableSensor(3, "Humidity", 5.0, datetime(2021, 10, 31))

#%%
# Create a list of sensors that should be sorted by calibration date
my_sensors = [sensor_1, sensor_2, sensor_3]
my_sensors.sort()

for my_sensor in my_sensors:
    print(my_sensor)

Sensor 'Humidity' (ID: 3) was calibrated on 31.10.2021.
Sensor 'Temperature' (ID: 1) was calibrated on 01.01.2022.
Sensor 'Pressure' (ID: 2) was calibrated on 17.08.2023.


## 🤓 Sortieren nach anderen Kriterien
Jetzt werden unsere Sensoren immer nach dem Kalibrierungsdatum sortiert.

In [20]:
# By default sensor can now only be sorted by the calibration date.
# If that is not desired we have to specified a sorting function to override the default one

my_sensors.sort(key=lambda x: x.id)

for my_sensor in my_sensors:
    print(my_sensor)

Sensor 'Temperature' (ID: 1) was calibrated on 01.01.2022.
Sensor 'Pressure' (ID: 2) was calibrated on 17.08.2023.
Sensor 'Humidity' (ID: 3) was calibrated on 31.10.2021.


## Serialisierung unserer Sensoren im JSON-Format
Im Austausch mit anderen Systemen (meist Web-Servern) werden Austauschformate für Daten benötigt. Ein gängiges dieser Formate ist JSON, welches in seiner Struktur Python-Dictionaries stark ähnelt.

In [21]:
sensor_4 = SortableSensor(4, "Acceleration", 9.81, None)
sensor_4.__dict__

{'id': 4,
 'sens_type': 'Acceleration',
 'sensitivity': 9.81,
 'calibration_date': None,
 '_Sensor__last_measurement': 0.0,
 'measurement_history': []}

Solange wir nur primitive Datentypen verwenden, können wir unsere Sensoren einfach in JSON umwandeln. In diesem Fall ist das Kalibrierungsdatum nicht gesetzt und wir laufen in keine Probleme.

In [22]:
json_string = json.dumps(sensor_4.__dict__, indent=2)
print(json_string)

{
  "id": 4,
  "sens_type": "Acceleration",
  "sensitivity": 9.81,
  "calibration_date": null,
  "_Sensor__last_measurement": 0.0,
  "measurement_history": []
}


Wir wollen nun unsere ganze Liste an Sensoren in ein JSON-Format bringen. 

Hierbei tritt aber ein **Fehler** auf, da das Kalibrierungsdatum der anderen drei Sensoren nicht in JSON umgewandelt werden kann. Wir müssen also eine Möglichkeit finden, wie wir unser Kalibrierungsdatum in ein JSON-Format umwandeln können.

In [23]:
# Serialize object to json string --> uses similar structure to a dict
json_string = json.dumps(my_sensors, indent=2)
print(json_string)

TypeError: Object of type SortableSensor is not JSON serializable

Wir könnten, wie oben wieder das DateTime-Object selbst anpassen. Das ist aber nicht immer möglich, da wir nicht immer Zugriff auf die Klasse haben, die wir serialisieren wollen. 

Um das `calibration_date` Attribut unserer Klasse serialiseren zu können, schreiben wir uns eine Hilfsfunktion (einen Serializer) um das `datetime`-Objekt in ein `dict`-Objekt zu verwandeln.

Diese könnnen wir dann in die `json.dumps()`-Funktion übergeben, um unser Objekt in ein JSON-String umzuwandeln. Die als `default` übergebene Funktion wird für jedes Objekt aufgerufen, das nicht direkt in ein JSON-Objekt umgewandelt werden kann.

In [24]:
def json_default(obj):
    """
    Define how to serialize objects
    Datetime objects will be serialized as dict with year, month, day
    All other objects by using their __dict__ attribute

    Args:
        value (Any): The object to serialize
    """
    if isinstance(obj, datetime):
        #return dict(year=obj.year, month=obj.month, day=obj.day)
        return {"year" : obj.year, "month" : obj.month, "day" : obj.day}
    else:
        return obj.__dict__

# Serialize object to json string --> uses similar structure to a dict
json_string = json.dumps(my_sensors, default=json_default, indent=2)
print(json_string)

[
  {
    "id": 1,
    "sens_type": "Temperature",
    "sensitivity": 1.0,
    "calibration_date": {
      "year": 2022,
      "month": 1,
      "day": 1
    },
    "_Sensor__last_measurement": 0.0,
    "measurement_history": []
  },
  {
    "id": 2,
    "sens_type": "Pressure",
    "sensitivity": 3.0,
    "calibration_date": {
      "year": 2023,
      "month": 8,
      "day": 17
    },
    "_Sensor__last_measurement": 0.0,
    "measurement_history": []
  },
  {
    "id": 3,
    "sens_type": "Humidity",
    "sensitivity": 5.0,
    "calibration_date": {
      "year": 2021,
      "month": 10,
      "day": 31
    },
    "_Sensor__last_measurement": 0.0,
    "measurement_history": []
  }
]


Nun können wir eine JSON-Datei aus diesen Daten erstellen.

In [25]:
# Open file and write json string to it
with open('./sensor.json', "w") as f:
    f.write(json_string)